# Using Logistic Regression for Review Classification

## Loading and Pre-Processing The Data

In [2]:
import pandas as pd

In [3]:
df = pd.read_json('../checkpoint-1/processed_data.json', orient='records', lines=True)

df.head()

,category,rating,label,text_,text_processed,text_tokenized,text_embeddings
0,Home and Kitchen,5,CG,"Love this! Well made, sturdy, and very comfor...",love well made sturdy comfortable love itvery ...,"[love, well, made, sturdy, comfortable, love, ...","[-0.06552676860000001, 0.0019102807000000001, ..."
1,Home and Kitchen,5,CG,"love it, a great upgrade from the original. I...",love great upgrade original ive mine couple years,"[love, great, upgrade, original, ive, mine, co...","[-0.11993740500000001, 0.0896221548, 0.0629631..."
2,Home and Kitchen,5,CG,This pillow saved my back. I love the look and...,pillow saved back love look feel pillow,"[pillow, saved, back, love, look, feel, pillow]","[-0.0531752855, 0.0186075028, 0.0380044468, 0...."
3,Home and Kitchen,1,CG,"Missing information on how to use it, but it i...",missing information use great product price,"[missing, information, use, great, product, pr...","[0.007157366300000001, 0.0636947528, 0.0084662..."
4,Home and Kitchen,5,CG,Very nice set. Good quality. We have had the s...,nice set good quality set two months,"[nice, set, good, quality, set, two, months]","[-0.0369832478, 0.011753053400000001, 0.041596..."


In [4]:
df = df.dropna()

In [5]:
X = df['text_embeddings'].tolist()
y = df['label'].tolist()

len(X), len(y)

(40408, 40408)

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()

y_encoded = lb.fit_transform(y).reshape(-1) # flattening the array

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, train_size=0.8)

In [ ]:
print(f"Num Samples in Training Set - {len(X_train)}, {len(y_train)}")
print(f"Num Samples in Testing Set  - {len(X_test)}, {len(y_test)}")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mmscaler = MinMaxScaler()

X_train_scaled = mmscaler.fit_transform(X_train)
X_test_scaled = mmscaler.transform(X_test)

## Getting A Baseline Score

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

logreg = LogisticRegression(max_iter=250) # Using the default hyperparameters

logreg.fit(X_train_scaled, y_train)

y_preds_baseline = logreg.predict(X_test_scaled)

print(f"Classification Report for Baseline Model - \n {classification_report(y_test, y_preds_baseline)}")

## Performing Hyperparameter Tuning

In [14]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'penalty': ['l1', 'l2'], 
    'C': [0.01, 0.1, 1, 10, 100],          
    'solver': ['liblinear', 'saga'],       
    'max_iter': [250, 500, 1000]           
}

logreg = LogisticRegression()

random_search = RandomizedSearchCV(estimator=logreg, param_distributions=param_grid, cv=5, scoring='accuracy', verbose=3, n_jobs=-1, n_iter=5)

random_search.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.
